# Facial Atributes Recognition Model based on MobileNetV2
This code creates a model to do facial atributes recognition which then will be used as a pretained model for a facial recognition model. 
This Conv2D model uses MobileNetV2 due to its lightness and rapidy during training. It was revised and discovered in **Vafaa Sukkar** and **Ergun Ercelebi** article **"A Real-time Face Recognition Based on MobileNetV2 Model"**. It can be found in https://ineseg.org/wp-content/uploads/2023/08/4.pdf

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os 

import optuna
import wandb
import gc

2025-11-29 11:31:27.237785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 11:31:27.237811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 11:31:27.238698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 11:31:27.244024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")
    
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow is using the GPU 
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-11-29 11:31:37.012625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-29 11:31:37.021053: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-29 11:31:37.022607: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
gc.collect()
tf.keras.backend.clear_session()

In [4]:
from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, (160, 160))
    return img, label

def make_dataset(paths, labels, batch_size = 32, shuffle = False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [6]:
# Se cargan los datos
# Se debe de realizar una transformación en los datos, convirtiéndolos de -1/1 a 0/1:

ds = pd.read_csv("/tf/Face-Recognition/CelebA/list_attr_celeba.txt", sep = r"\s+", skiprows = 1)
ds.iloc[:, 0:40] = (ds.iloc[:, 0:40] == 1).astype("int32")
ds.head()
# ds.describe()
# (ds == -1).sum()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,0,1,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
000002.jpg,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
000003.jpg,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
000004.jpg,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
000005.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [7]:
img_dir = "/img_align_celeba"

ds["image_path"] = ds.index.map(lambda x: os.path.join(img_dir, x))
ds.reset_index(inplace = True)
ds.rename(columns = {"index" : "image"}, inplace = True)
ds.head()

,image,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_path
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,1,1,0,1,0,1,0,0,1,/img_align_celeba/000001.jpg
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,/img_align_celeba/000002.jpg
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,/img_align_celeba/000003.jpg
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,/img_align_celeba/000004.jpg
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,/img_align_celeba/000005.jpg


In [8]:
df_split = pd.read_csv(
    "/tf/Face-Recognition/CelebA/Eval/list_eval_partition.txt", 
    sep = r"\s+",
    names = ["image", "partition"])

ds = ds.merge(df_split, on = "image")
ds.head()

,image,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_path,partition
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,1,/img_align_celeba/000001.jpg,0
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,/img_align_celeba/000002.jpg,0
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,/img_align_celeba/000003.jpg,0
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,1,0,1,0,1,1,0,1,/img_align_celeba/000004.jpg,0
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,/img_align_celeba/000005.jpg,0


In [9]:
os.path.exists(ds["image_path"].iloc[0])

True

In [10]:
df_train = ds[ds["partition"] == 0]
df_val   = ds[ds["partition"] == 1]
df_test  = ds[ds["partition"] == 2]

In [11]:
train_ds = make_dataset(
    df_train["image_path"].values, 
    df_train.iloc[:, 1:41].values,
    shuffle = True
)

val_ds = make_dataset(
    df_val["image_path"].values,
    df_val.iloc[:, 1:41].values,
    shuffle = False
)

test_ds = make_dataset(
    df_test["image_path"].values,
    df_test.iloc[:, 1:41].values,
    shuffle = False
)

2025-11-29 11:31:38.544105: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-29 11:31:38.545259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-29 11:31:38.546143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
df_train_small = df_train.sample(frac = 0.15, random_state = 5)
df_val_small = df_val.sample(frac = 0.15, random_state = 5)
df_test_small = df_test.sample(frac = 0.15, random_state = 5)

train_ds_small = make_dataset(
    df_train_small["image_path"].values,
    df_train_small.iloc[:, 1:41].values,
    shuffle = True,
)

val_ds_small = make_dataset(
    df_val_small["image_path"].values,
    df_val_small.iloc[:, 1:41].values,
)

test_ds_small = make_dataset(
    df_test_small["image_path"].values,
    df_test_small.iloc[:, 1:41].values,
)

In [13]:
MobileNetV2 = keras.applications.MobileNetV2(
    weights = "imagenet",
    input_shape = (160, 160, 3),
    include_top = False)

In [14]:
MobileNetV2.trainable = False

In [15]:
lr = 1e-4
optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr) 

In [16]:
inputs = keras.Input(shape = (160, 160, 3))
x = keras.applications.mobilenet_v2.preprocess_input(inputs)
x = MobileNetV2(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.15)(x)
x = layers.Dense(64)(x)
x = layers.ReLU()(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(40, activation = "sigmoid", dtype = "float32")(x)
model = keras.Model(inputs, outputs)

model.compile(loss = "binary_crossentropy",
              optimizer = optimizer,
              metrics = [tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 160, 160, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 160, 160, 3)       0         
 da)                                                             
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                     

In [17]:
early_stopping = EarlyStopping(monitor = "val_binary_accuracy", patience = 10, restore_best_weights = True)
lr_reduction = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 7)

In [18]:
wandb.init(
        project = "Conv2D-MobileNetV2-Based-Trials-Exp-Series1.0",
        name = "Trial_1_FullSet",
        reinit = True,
        config = {
            "activation": "leaky_relu, relu",
            "n_layers": 3,
            "learning_rate": lr,
            "optimizer": "RMSProp"
        }
    )

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [19]:
history = model.fit(
    train_ds, 
    validation_data = val_ds,
    epochs = 200,
    verbose = 1, 
    callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping, lr_reduction]
        )

Epoch 1/200


2025-11-29 11:31:44.261561: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-11-29 11:31:47.631180: I external/local_xla/xla/service/service.cc:168] XLA service 0x75f12c1cf5c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-29 11:31:47.631203: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-11-29 11:31:47.635554: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764415907.668833   67185 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5087/5087 [==============================] - 186s 35ms/step - loss: 0.3418 - binary_accuracy: 0.8482 - auc: 0.8782 - val_loss: 0.2933 - val_binary_accuracy: 0.8701 - val_auc: 0.9139 - lr: 1.0000e-04
Epoch 2/200
5087/5087 [==============================] - 227s 45ms/step - loss: 0.3021 - binary_accuracy: 0.8658 - auc: 0.9084 - val_loss: 0.2829 - val_binary_accuracy: 0.8747 - val_auc: 0.9204 - lr: 1.0000e-04
Epoch 3/200
5087/5087 [==============================] - 324s 64ms/step - loss: 0.2933 - binary_accuracy: 0.8697 - auc: 0.9142 - val_loss: 0.2784 - val_binary_accuracy: 0.8766 - val_auc: 0.9231 - lr: 1.0000e-04
Epoch 4/200
5087/5087 [==============================] - 176s 35ms/step - loss: 0.2885 - binary_accuracy: 0.8717 - auc: 0.9173 - val_loss: 0.2755 - val_binary_accuracy: 0.8777 - val_auc: 0.9247 - lr: 1.0000e-04
Epoch 5/200
5087/5087 [==============================] - 177s 35ms/step - loss: 0.2852 - binary_accuracy: 0.8732 - auc: 0.9194 - val_loss: 0.2735 - val_binary_accuracy:

In [20]:
model.save("Conv2D-MobileNetV2-Based2.keras") 

In [21]:
tf.keras.backend.clear_session()
wandb.finish()
gc.collect()

batch/auc,▁▂▄▄▄▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
batch/batch_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇██
batch/binary_accuracy,▁▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇███████████████
batch/learning_rate,█████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
batch/loss,█▆▆▅▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/auc,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████████
epoch/binary_accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
epoch/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,██████████████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁▁
epoch/loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+3,...


1287

### Multi-label (CelebA) considerations

CelebA is **multi-label** each image has 40 independent binary attributes. Important implications:

- **BinaryAccuracy:** When using a single BinaryAccuracy() metric in model.compile, it computes accuracy element-wise over the whole output tensor. Concretely:

    - If batch size = 32 and 40 labels, each update_state compares 32×40 = 1280 elements and returns fraction correct out of 1280.

    - This is typically more informative than “exact match accuracy” (which requires all 40 to be correct per image and is often near zero).

- **AUC for multi-label:** AUC can be evaluated:

    - **Per-class AUC:** Computes AUC separately for each of the 40 labels (which is recommended). This reveals which attributes the model predicts well and which it doesn’t.

    - **Micro AUC:** Treats every label-instance pair as separate samples (flatten all labels and predictions) and computes a single AUC across them (equivalent to element-wise pooling). This weights attributes by frequency.

    - **Macro AUC:** Averages per-class AUC equally (gives each attribute equal weight regardless of prevalence).

### Fine-Tunning

In [22]:
MobileNetV2.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[

In [23]:
MobileNetV2.trainable = True

for layer in MobileNetV2.layers[:-10]:
  layer.trainable = False

In [24]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-6)

In [25]:
model.compile(loss = "binary_crossentropy",
              optimizer = optimizer,
              metrics = [tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()])

In [26]:
wandb.init(
        project = "Conv2D-MobileNetV2-Based-Trials-Exp-Series1.0",
        name = "FineTunning_1_FullSet",
        reinit = True,
        config = {
            "activation": "leaky_relu, relu",
            "n_layers": 3,
            "learning_rate": lr,
            "optimizer": "RMSProp"
        }
    )

In [27]:
history = model.fit(
    train_ds, 
    validation_data = val_ds,
    epochs = 1,
    verbose = 1)

2025-11-29 14:41:02.891493: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1007.56MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-29 14:41:02.926931: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1007.56MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-29 14:41:03.194566: W external/local_tsl/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur g

5087/5087 [==============================] - 189s 36ms/step - loss: 0.3252 - binary_accuracy: 0.8557 - auc: 0.8926 - val_loss: 0.2926 - val_binary_accuracy: 0.8700 - val_auc: 0.9140


In [28]:
model.save("Conv2D-MobileNetV2-Based-Fine-Tunned2.keras") 

In [29]:
tf.keras.backend.clear_session()
wandb.finish()
gc.collect()

644

In [30]:
model_ev = keras.models.load_model("Conv2D-MobileNetV2-Based-Fine-Tunned2.keras")

loss, binary_accuracy, auc_1 = model_ev.evaluate(test_ds, verbose = 1)

print(f"Test Loss: {loss:}")
print(f"Test Accuracy: {binary_accuracy:}")

624/624 [==============================] - 20s 31ms/step - loss: 0.2991 - binary_accuracy: 0.8646 - auc: 0.9130
Test Loss: 0.29912495613098145
Test Accuracy: 0.8646227717399597
